<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/Claude/Claude102_SQLQueries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install anthropic -q
!pip install supabase -q
!pip install pyodbc -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 5.4 MB/s eta 0:00:00


In [6]:
import anthropic

from google.colab import userdata
claude_api_key=userdata.get('Claude_api_key')

client = anthropic.Anthropic(api_key=claude_api_key)

In [7]:
#
# Retrieve a file from Github Repository

def get_ChinookDB_ER_from_github(file_url):

  # Send a GET request to the file URL
  response = requests.get(file_url)

  # Check if the request was successful
  if response.status_code == 200:
      # Get the content of the file
      file_content = response.text
      return file_content
  else:
      return(f'Failed to retrieve the file: {response.status_code}')

In [11]:
from supabase import create_client, Client

In [104]:
from supabase import create_client, Client
from sqlalchemy.sql import text

from sqlalchemy import create_engine
# Test connection to Supabase using Supabase client

# Make sure the connection string is a proper string within quotes
supabase_conn_str = userdata.get('supabase_conn_str')
#supabase_conn_str = "postgresql+psycopg2://[USERID:PASSWORD]@[HOSTNAME:PORT_NUMBER]/[DATABASE_NAME]"


# Use the connection string with SQLAlchemy's create_engine function
db_engine = create_engine(supabase_conn_str)
sql_conn = db_engine.connect()

# test the connection
result = sql_conn.execute(text('select * from "Customer";'))
df = pd.DataFrame(result)
df.head(3)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


In [62]:
import requests
chinookDB_ER_from_github = get_ChinookDB_ER_from_github("https://raw.githubusercontent.com/cooolbabu/GoogleGemini101/main/OpenAI/ChinookDB_ER.txt")
# Markdown(chinookDB_ER_from_github) ## uncoment to view file from Github

## Build the prompt
dba_instructions="""
1. You are a Data Architect for ChinookDB.
2. An entiry-relationship diagram for ChinookDB, in the form of a Mermaid.js script is provide to you
3. You will use this ER diagram to deliver SQL queries.
4. Database dialect is PostgreSQL
5. Use quotes for database object names in SQL statements
6. Give your response in JSON format having two attributes. SQL and Explanation
7. SQL attribute will have the value of the generated SQL code
8. Explanation attribute will have the value of the explanation
9. Remove all introductory messages.

"""

In [101]:
Prompt = dba_instructions + chinookDB_ER_from_github + "Get total sales for each genre by quarter for each year "
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": Prompt}
    ]
)


In [102]:
# Parse the JSON string
parsed_json = json.loads(message.content[0].text, strict=False)

# Accessing elements
sql_query = parsed_json["SQL"]
explanation = parsed_json["Explanation"]

print("SQL Query:", sql_query)
print("Explanation:", explanation)

SQL Query: SELECT 
    g."Name" AS "Genre",
    EXTRACT(YEAR FROM i."InvoiceDate") AS "Year",
    EXTRACT(QUARTER FROM i."InvoiceDate") AS "Quarter",
    SUM(il."Quantity" * il."UnitPrice") AS "Total Sales"
  FROM "Genre" g
  JOIN "Track" t ON g."GenreId" = t."GenreId"
  JOIN "InvoiceLine" il ON t."TrackId" = il."TrackId"
  JOIN "Invoice" i ON il."InvoiceId" = i."InvoiceId"
  GROUP BY 
    g."Name",
    EXTRACT(YEAR FROM i."InvoiceDate"),
    EXTRACT(QUARTER FROM i."InvoiceDate")
  ORDER BY
    g."Name",
    EXTRACT(YEAR FROM i."InvoiceDate"),
    EXTRACT(QUARTER FROM i."InvoiceDate");
Explanation: This query retrieves the total sales for each genre by quarter for each year. Here's how it works:

1. The FROM clause starts with the "Genre" table, which is joined to the "Track" table using the "GenreId" foreign key.

2. The "Track" table is then joined to the "InvoiceLine" table using the "TrackId" foreign key.

3. The "InvoiceLine" table is joined to the "Invoice" table using the "Invoi

In [105]:
result = sql_conn.execute(text(sql_query))
df = pd.DataFrame(result)
df.head(3)

,Genre,Year,Quarter,Total Sales
0,Alternative,2022,1,5.94
1,Alternative,2023,2,3.96
2,Alternative,2024,4,3.96


In [ ]:
pd.set_option('display.max_rows', None)
df.sort_values(by=['Year', 'Quarter', 'Genre'])